# HW 7

## Problem 1

In [82]:
import numpy as np
from scipy.integrate import simpson as simps

# EVERYTHING IN CGS
sigma = 5.670e-5 # cgs
c = 3e10 # cgs
h = 6.626e-27 # cgs
k = 1.3807e-16
M_solar = 1.989e33 # g
L_solar = 3.839e33
R_solar = 6.95e10
m_e = 9.1e-28
m_p = 1.67e-24

ev_to_erg = 1.6e-12


In [267]:
def L(M):
    return L_solar * (M/M_solar)**3.5

def R(M):
    return R_solar * (M/M_solar)**0.8

def T(M):
    return (L(M) / (4*np.pi*R(M)**2*sigma))**0.25

def n(nu,M):
    frac1 = 8 * np.pi * nu**3 * c**-2

    exponent = h*nu/(k*T(M))
    exponent = np.clip(exponent, 1e-10, 700)
    frac2 = (np.exp(exponent) - 1)**-1
    return frac1*frac2

def salpeter(M):
    # NOTE: unnormalized
    return M**-2.35

def B_nu(nu,T):
    frac1 = 2*h*nu**3*c**-2

    exponent = h*nu/(k*T)
    exponent = np.clip(exponent, 1e-10, 700)
    frac2 = (np.exp(exponent)-1)**-1

    return frac1*frac2

def N_gamma_per_stellar_mass(M):
    nu_cutoff = 13.6*ev_to_erg/h
    nus = np.logspace(np.log10(nu_cutoff),18,1000)
    integrand = B_nu(nus,T(M))/(h*nus) * np.pi * 4*np.pi*R(M)**2
    return simps(integrand,nus)

def t_star(M):
    # Very rough approximation for massive stars
    return 1e10 * (M_solar/M)**(2.5) * 3.14e7  # in seconds

def Q():
    Ms = np.linspace(.01,100,500) * M_solar
    numerator_integrand = salpeter(Ms) * Ms * [N_gamma_per_stellar_mass(M) for M in Ms] * t_star(Ms)
    numerator = simps(numerator_integrand, Ms)

    denom_integrand = salpeter(Ms) * Ms
    denom = simps(denom_integrand, Ms)
    return numerator/denom

In [268]:
print(f"Q = {Q():.2e} ")

Q = 7.84e+60 


In [291]:
m_b = (m_p + m_e)/M_solar
F = 1/(Q() * m_b)
F_percent = 100*F
print(f"F = {F_percent:.3f}%\n")

F = 0.015%



In [290]:
print("Accounting for recombination and escape fraction gives:")
photons_per_HI = 3
escape_frac = .2
F_corrected = F_percent*photons_per_HI/escape_frac
print(f"F = {F_corrected:.2f}%")

Accounting for recombination and escape fraction gives:
F = 0.23%


In [368]:
rho_star = 10**6.7
h_cosmo = .7
omega_b = .022/(h_cosmo**2)
rho_crit = 2.775*h_cosmo**2 * 1e11 
z=4.25
rho_b = omega_b*rho_crit
print(f"{rho_b:.2e}")

6.10e+09


In [369]:
print(f"rho_*/rho_b = {rho_star/rho_b * 100:.3f}%")

rho_*/rho_b = 0.082%


## Problem 2

In [333]:
def c_s(T, gamma = 5/3, mu=.6):
    return np.sqrt(gamma*k*T/(mu*m_p))

print(f"c_s = {c_s(1e4):.2e} cm/s")

c_s = 1.51e+06 cm/s


In [341]:
G = 6.67e-8
R_crit = 2*G*M_solar/c_s(1e4)**2 / 1.496e+13
print(f"{R_crit:.2f} AU")

7.73 AU


## Problem 3

In [ ]:
L_star = 6e5*L_solar
T_star = 4e4
R_star = np.sqrt(L_star/(4*np.pi*sigma*T_star**4))

def Q_0(T_star, R_star):
    nu_cutoff = 13.6*ev_to_erg/h
    nus = np.logspace(np.log10(nu_cutoff),18,1000)
    integrand = B_nu(nus,T_star)/(h*nus) * np.pi * 4*np.pi*R_star**2
    return simps(integrand,nus)

print(f"Q_0 = {Q_0(T_star,R_star):.2e} photons/sec")

Q_0 = 3.19e+49 photons/sec


In [354]:
def r_s(n_H, T_star, R_star, alpha_B=2e-13):
    r = ((3*Q_0(T_star,R_star))/(4*np.pi*n_H**2*alpha_B))**(1/3)
    return r/3.086e18 # output in pc

print(f"r_s = {r_s(1e2, T_star, R_star):.2f} pc")

r_s = 5.06 pc


In [363]:
M_1450 = -25
f_1450 = 10**((M_1450+48.6)/(-2.5))
L_1450 = f_1450 * 4 * np.pi * (10 * 3e18)**2
print(f"L_1450 = {L_1450:.2e} erg/s/Hz")

L_1450 = 4.11e+30 erg/s/Hz


In [365]:
def N_gamma_dot():
    def L(nu):
        nu_1450 = c/(1450*1e-8)
        return L_1450*(nu/nu_1450)**-1.4

    
    nu_cutoff = 13.6*ev_to_erg/h
    nus = np.logspace(np.log10(nu_cutoff),18,1000)

    integrand = L(nus) / (h*nus)

    return simps(integrand,nus)

N_gamma_dot()

print(f"N_gamma = {N_gamma_dot():.2e} photons/s")

N_gamma = 2.32e+56 photons/s


In [376]:
z = 7
h_cosmo = .7
omega_b = .022/(h_cosmo**2)
rho_crit = 1.879 * h_cosmo**2 * 1e-29
rho_b0 = omega_b * rho_crit
rho_b = rho_b0 * (1+z)**3
n_H = (rho_b * .75) / (m_e+m_p)
print(f"n_H = {n_H:.2e}")

n_H = 9.49e-05


In [386]:
def t_Q(r_s):
    num = 4*np.pi*n_H*r_s**3
    denom = 3*N_gamma_dot()
    return num/denom

r_s = 1 * 3e18 * 1e6 # convert Mpc to cm
t = t_Q(r_s) 
t_Myrs = t_Q(r_s) / (3.14e7 * 1e6) # 
print(f"{t_Myrs:.2f} Myrs")

1.47 Myrs
